In [ ]:
import os
import pandas as pd

In [ ]:
all_files = [os.path.join('../data/waitrose', file) for file in os.listdir('../data/waitrose') 
             if file.endswith('.csv')]

In [ ]:
df = pd.concat((pd.read_csv(file) for file in all_files))

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df[['item-price','product-size','price-per-unit']]

In [ ]:
df=df.drop_duplicates()

df= df.drop(columns=['data-product-name',
                     'data-product-type',
                     'data-product-index'])


In [ ]:
df.info()

In [ ]:
df=(df.rename(columns={
    'data-product-id':'id',
    'data-product-on-offer':'offer',
    'product-name':'name',
    'product-page':'page',
    'product-size':'size',
    'item-price':'item_price'

}))
df['id']=df['id'].astype('int32')

In [ ]:
df.info()

In [ ]:
df['item_price'].iloc[120:131]

In [ ]:
def clean_item_price(item_price:str):    
    if 'each est.' in item_price:
        item_price = item_price.replace(' each est.', "")
    if type(item_price)!=str:
        pass
    elif '£' in item_price:
        item_price = item_price.replace('£',"")
    elif 'p' in item_price:
        item_price = item_price.replace('p',"")
        item_price = '0.' + item_price

    if '-' in item_price:
        item_price = item_price.split('-')[0]
    return float(item_price)


In [ ]:
df['item-price-fixed']=df['item_price'].astype(str).apply(clean_item_price)

In [ ]:
df['item-price-fixed']

In [ ]:
selected_rows =df['item_price'].astype(str).str.contains('-')
df.loc[selected_rows,'size']=df.loc[selected_rows,'size'].str.split("-").str.replace('(','')

In [ ]:
df['item_price']=df['item-price-fixed']

In [ ]:
df

In [ ]:
df['item_price']=df['item-price-fixed']
df.drop(columns=['item-price-fixed'], inplace= True)

In [ ]:
df

In [ ]:
df.groupby('category')['item_price'].describe().reset_index()

In [ ]:
plot_df=df.groupby('category')['item_price'].describe().reset_index()

In [ ]:
plot_df.rename(columns={
    "25%":"Q1",
    "50%":"median",
    "75%":"Q3"
},inplace=True)

In [ ]:
plot_df

In [ ]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()
import numpy as np

In [ ]:
# This configures what shows up when you hover your mouse over the plot.
tooltip_setup = (
    layer_tooltips()
        .line('@category')
        .line('[@Q1 -- @median -- @Q3]')
        .format('@Q1', '£ {.2f}')
        .format('@median', '£ {.2f}')
        .format('@Q3', '£ {.2f}')
)

g = (
    # Maps the columns to the aesthetics of the plot.
    ggplot(plot_df, aes(y='category', x='median', xmin='Q1', xmax='Q3', fill='category')) +

    # GEOMS

    # Add a line range that 'listens to' columns informed in `ymin` and `ymax` aesthetics
    geom_linerange(size=1, alpha=0.75, tooltips=tooltip_setup) +

    # Add points to the plot (listen to `x` and `y` and fill aesthetics)
    geom_point(size=3, stroke=1, shape=21, tooltips=tooltip_setup) +

    # SCALES

    # Remove the legend (we can already read the categories from the y-axis)
    scale_fill_discrete(guide='none') +

    # Specify names for the axes
    scale_y_continuous(name="Categories\n(from largest to smallest median)", expand=[0.05, 0.05]) +
    scale_x_continuous(name="Price (£)", expand=[0., 0.05], format='£ {.2f}', breaks=np.arange(0, 20, 2.5)) +

    # LABELS
    # It's nice when the plot tells you the key takeaways
    labs(title='"Beer, Wine & Spirits" has the highest median price for individual items',
         subtitle="Dots represent the median price, bars represent the 25th and 75th percentiles") +
    theme(axis_text_x=element_text(size=15),
        axis_text_y=element_text(size=17),
        axis_title_x=element_text(size=20),
        axis_title_y=element_text(size=20),
        plot_title=element_text(size=19, face='bold'),
        plot_subtitle=element_text(size=18),
        legend_position='none') +
    ggsize(1000, 500)

)

g